In [1]:
def binary_to_decimal(binary_string):
    if binary_string[0] == '1':  # Check if the number is negative
        # Calculate the two's complement by subtracting 2^n
        return int(binary_string, 2) - 2**len(binary_string)
    else:
        return int(binary_string, 2)

def decimal_to_binary(decimal_number, bit_width=8):
    if decimal_number < 0:
        # Calculate the two's complement by adding 2^n
        binary_string = bin(decimal_number + 2**bit_width)[2:]
    else:
        binary_string = bin(decimal_number)[2:]

    # Ensure the binary string has the correct length (bit_width)
    return binary_string.zfill(bit_width)

In [2]:
# Dictionary with instruction op codes
mnemo_op_code = {
    "LDA": "0000",
    "ADD": "0001",
    "SUB": "0010",
    "OUT": "1110",
    "HLT": "1111"
}

op_code_mnemo = {
    '0000' : 'LDA', 
    '0001' : 'ADD',
    '0010' : 'SUB',
    '1110' : 'OUT',
    '1111' : 'HLT'
}

def convert_mnemo_op_code(instruction):
    op_code = mnemo_op_code.get(instruction, "N/A")
    return op_code

# Function load
def old_load(file_path, ram_instance):

    # Open the file in reading
    with open(file_path, "r") as file:
        # Read each line
        for line_number, line in enumerate(file):
            # Delete blanks
            line = line.strip()

            if isalpha(line[0]):
                # Split the line into instruction and hexadecimal number
                parts = line.split()
                instruction = parts[0]
                hex_number = parts[1]

                # Convert instruction to op code
                op_code = convert_mnemo_op_code(instruction)

                # Convert hexadecimal number to binary and take the last 4 digits
                binary_number = bin(int(hex_number, 16))[2:].zfill(4)[-4:]

                # Combine op code and binary number
                combined_data = op_code + binary_number

            else:
                combined_data = bin(int(hex_number, 16))[2:].zfill(8)[-8:]
            # Append combined data to RAM using line number as address
            ram_instance.write(combined_data, line_number)

# Create an instance of the RAM class
#my_ram = RAM()

# Call the load function with the file path and the RAM instance
#load(r"file.txt", my_ram)

#my_ram.read(1)


In [3]:
class Clock:
	def __init__(self, cpu):
		self.state=0
		self.on=True

	def advance(self):
		print(cpu)
		self.state+=1

	def __str__(self):
		return self.state

class RingCounter():
	def __init__(self, n_time_states):
		self.state=0
		self.n_time_states=n_time_states

	def advance(self):
		self.state=(self.state+1)%self.n_time_states
	
	def read(self):
		return self.state
		
	def __str__(self):
		return self.state

In [4]:
class Register:
    """
    Three 8-bit registers: A, B, IR. A and B are general-purpose, IR is the instruction register (store the current instruction that's being executed).
    Three numpy array of fixed size
    """

    def __init__(self):
        self.state = ''
    
    # Return the state of the register
    def read(self):
        return self.state
    
    # Overwrite the state of the register
    def write(self, data):
        self.state = data
    
    def __str__(self):
        return f"State: {self.state}"

class ByteRegister(Register):
    def __init__(self):
        self.state = '0'*8

    def __str__(self):
        return f"State: {self.state}"

class NibbleRegister(Register):
    def __init__(self):
        self.state = '0'*4
    
    def __str__(self):
        return f"State: {self.state}"
    
class RAM(Register):
    """
    RAM stores both the program and the data. 16 addresses of 8 bits each
    """

    def __init__(self):
        # List of 8-bits registers
        self.state = [ByteRegister() for _ in range(16)]

    # Write on the address of the RAM
    def write(self, data, address):
        # Data validation
        assert type(data) is str, 'data type error'
        assert len(data) == 8, 'size error'

        self.state[address].write(data)
    
    # Read the address of the RAM
    def read(self, address):
        return self.state[address].read()
    
    def __str__(self):
        return "\n".join([f"Address {i}: {register}" for i, register in enumerate(self.state)])



class InstructionRegister(ByteRegister):
    """
    Register where you can access the first and the second nibbles 
    """

    def readOpCode(self):
        return self.read()[:4]
    
    def readOperand(self):
        return self.read()[4:]

    def __str__(self):
        return f"OpCode: {self.readOpCode()}\nOperand: {self.readOperand()}"

class ProgramCounter(NibbleRegister):
    """
    4-bits program counter
    """

    # Update the counter
    def advance(self):       
        # Get the decimal value
        value = int(self.state, 2)
        # Update the counter mod 2**4
        value = (value + 1)%2**4
        # Convert the new value into a 4-bits string
        string = bin(value)[2:].zfill(4)

        self.state = string

    def __str__(self):
        return f"Value: {int(self.state, 2)}"



class Flag(Register):
    def __init__(self, reg):
        self.state = 0
        self.reg=reg
    
    
    def read(self):
        if(binary_to_decimal(self.reg.read())<0):
            self.state=1
        else:
            self.state=0
    def __str__(self):
        self.read()
        return f"State: {self.state}"



class ALU(ByteRegister):
    """
    The logic of the CPU
    """
    def __init__(self, aRegister, bRegister):
        self.a = aRegister
        self.b = bRegister
        self.state='0'*8
        
    def read(self, mode):
        value_a, value_b=binary_to_decimal(self.a), binary_to_decimal(self.b)
        if(mode=='add'):
            result=value_a+value_b
        elif(mode=='sub'):
            result=value_a-value_b
        self.state=result[0:8]
        return decimal_to_binary(result[0:8])
        
    def __str__(self):
        self.read()
        return self.state


In [5]:

#HELPER FUNCTIONS

#helper function for moving stuff between registers

def bus(reg1, reg2, cpu):
	"""
	Copy data of reg1 into reg2
	"""
	cpu.advance_clock()
	reg2.write(reg1.read())

#MICRO-INSTRUCTIONS IMPLEMENTATION
	
#exemples of micro-instructions. Micro-instructions are functions of ONLY a cpu object:

def pc_to_mar(cpu):
	bus(cpu.programCounter,cpu.memoryAddressRegister,cpu)

"""RAM.read() HA BISOGNO DI INDICE!"""
def ram_to_a(cpu):
	address=binary_to_decimal(cpu.memoryAddressRegister.read())
	cpu.aRegister.write(cpu.ram.read(address))

def ram_to_b(cpu):
	address=binary_to_decimal(cpu.memoryAddressRegister.read())
	cpu.bRegister.write(cpu.ram.read(address))

def ram_to_ir(cpu):
	address=binary_to_decimal(cpu.memoryAddressRegister.read())
	cpu.instructionRegister.write(cpu.ram.read(address))

def ir_to_mar(cpu):
	cpu.memoryAddressRegister.write(cpu.instructionRegister.readOperand())

#def alu_to_a(cpu):
	#bus(cpu.alu, cpu.aRegister, cpu)

def alu_to_a_add(cpu):
	cpu.aRegister.write(cpu.alu.read('add'))

def alu_to_a_sub(cpu):
	cpu.aRegister.write(cpu.alu.read('sub'))

def no_op(cpu):
	pass

def clock_off(cpu):
	cpu.clock.on=False
	

#dict of lists of functions, each list is addressed by an opcode
#corresponding to a macroinstruction available to the cpu,
#and contains functions that corresponds to micro-instructions.
#Very easy to expand the set, and very readable.

macro_instructions={ 
	'LDA':[ir_to_mar, ram_to_a, no_op],
	'ADD':[ir_to_mar, ram_to_b, alu_to_a_add],
	'SUB':[ir_to_mar, ram_to_b, alu_to_a_sub],
	'HLT':[clock_off, no_op, no_op],
}


In [30]:
class CPU:
	def __init__(self): #inizializza a zero i clock, program counter e registri
		self.clock=Clock(self)
		self.ringCounter=RingCounter(6)
		self.programCounter=ProgramCounter()
		self.memoryAddressRegister=NibbleRegister()
		self.ram=RAM()
		self.aRegister=ByteRegister()
		self.bRegister=ByteRegister()
		self.instructionRegister=InstructionRegister()
		self.alu=ALU(self.aRegister,self.bRegister)
		self.flagA=Flag(self.aRegister)	



	def load(self, file_path):

		# Open the file in reading
		with open(file_path, "r") as file:
			# Read each line
			for line_number, line in enumerate(file):
				# Delete blanks
				line = line.strip()

				if line[0].isalpha():
					# Split the line into instruction and hexadecimal number
					parts = line.split()
					instruction = parts[0]
					hex_number = parts[1]

					# Convert instruction to op code
					op_code = convert_mnemo_op_code(instruction)

					# Convert hexadecimal number to binary and take the last 4 digits
					binary_number = bin(int(hex_number, 16))[2:].zfill(4)[-4:]

					# Combine op code and binary number
					combined_data = op_code + binary_number

				else:
					combined_data = bin(int(hex_number, 16))[2:].zfill(8)[-8:]
					print(bin(int(hex_number, 16)))
					print()
					# Append combined data to RAM using line number as address
				self.ram.write(combined_data, line_number)


	def advance_clock(self, how_much=1):
		for i in range(how_much):
			self.clock.advance()
			self.ringCounter.advance()
	

	# Execute the program 
	def run(self): #esegue il programma in ram
		self.clock.on=True
		while(self.clock.on):
			#fetch:
			pc_to_mar(self)
			self.advance_clock()
			ram_to_ir(self)
			self.advance_clock()
			self.programCounter.advance()
			self.advance_clock()
			
			while(self.clock.on):
		
				op_code=self.instructionRegister.readOpCode()
				time_state=self.ringCounter.read()
				macro_instructions[op_code_mnemo[op_code]][time_state-3](self)

                #questo esegue la micro-operazione dell'istruzione presente sull'ir,
                #corrispondente al time-state indicato sul microclock.

				self.advance_clock()
				if self.clock.state>10:
				
					self.clock.on = False
					break


	def __str__(self):
		snapshot = f"Clock: {self.clock.__str__()}\nRing Counter: {self.ringCounter.__str__()}\n Program Counter: {self.programCounter.__str__()}\n" \
                   f"Memory Address Register: {self.memoryAddressRegister.__str__()}\nA Register: {self.aRegister.__str__()}\n" \
                   f"B Register: {self.bRegister.__str__()}\nInstruction Register: {self.instructionRegister.__str__()}\n" \
                   f"Flag A: {self.flagA.__str__()}\nRAM:\n{self.ram.__str__()}\n"

		
		return snapshot

		

In [31]:
#esempio main
cpu= CPU()
cpu.load('file.txt')
print(cpu)
#cpu.run()


0b0

Clock: 0
Ring Counter: 0
 Program Counter: Value: 0
Memory Address Register: State: 0000
A Register: State: 00000000
B Register: State: 00000000
Instruction Register: OpCode: 0000
Operand: 0000
Flag A: State: 0
RAM:
Address 0: State: 00000010
Address 1: State: 11110000
Address 2: State: 00000000
Address 3: State: 00000000
Address 4: State: 00000000
Address 5: State: 00000000
Address 6: State: 00000000
Address 7: State: 00000000
Address 8: State: 00000000
Address 9: State: 00000000
Address 10: State: 00000000
Address 11: State: 00000000
Address 12: State: 00000000
Address 13: State: 00000000
Address 14: State: 00000000
Address 15: State: 00000000

